Create RabitMQ connection and channel

In [ ]:
import pika
hostName = 'localhost'
helloChannelName = 'Download'
helloRoutingKey = 'Download'

connection = pika.BlockingConnection(pika.ConnectionParameters(host=hostName))
channel = connection.channel()

channel.queue_declare(queue=helloChannelName, durable = True)

Send messages

In [ ]:
import json
import datetime
from json import JSONEncoder

class DownloadRequestMessage:
    Url = ""
    SourceId = ""
    TimeStamp = datetime.datetime.now()
    
    def __init__(self, url: str, sourceId: str, timeStamp: datetime):
        self.Url = url
        self.SourceId = sourceId
        self.TimeStamp = timeStamp

class DateTimeEncoder(JSONEncoder):
        #Override the default method
        def default(self, obj):
            if isinstance(obj, (datetime.date, datetime.datetime)):
                return obj.isoformat()

messageObj = DownloadRequestMessage(
    url = "https://techcrunch.com/2023/01/25/2022-european-edtech-report-smaller-rounds-and-fewer-deals-but-more-angel-activity/?guccounter=1",
    sourceId = "cloudamqp",
    timeStamp = datetime.datetime.now())

jsonMessage = json.dumps(messageObj.__dict__, indent=4, cls=DateTimeEncoder)

print(jsonMessage)

In [ ]:
channel.basic_publish(exchange='', routing_key = helloRoutingKey, body=jsonMessage)
print(jsonMessage)

Close connection. Cleanup resources.

In [ ]:
connection.close()